Backprop Ninja
https://www.youtube.com/watch?v=q8SA3rM6ckI


In [1]:
import random

import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from matplotlib.colors import Normalize

%matplotlib inline

In [2]:
# read in all the words
words = open("../makemore/names.txt", "r").read().splitlines()
(len(words), words[:5])

(32033, ['emma', 'olivia', 'ava', 'isabella', 'sophia'])

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)

In [4]:
# build the dataset

# context length: how many characters do we take to predict the next one?
block_size = 3


def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]  # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])  # 80%
Xdev, Ydev = build_dataset(words[n1:n2])  # 10%
Xte, Yte = build_dataset(words[n2:])  # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t, img=False):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{s:15s} | exact: {str(ex):5s} | approx: {str(app):5s} | maxdiff: {maxdiff}")

    if img or ((not ex) and (not app)):
        fig, (left, right) = plt.subplots(1, 2)

        norm = Normalize(vmin=min(dt.min(), t.grad.min()), vmax=max(dt.max(), t.grad.max()))  # fmt: skip

        left.imshow(dt.detach().numpy(), cmap="coolwarm", norm=norm)
        left.set_xlabel("dt")

        image = right.imshow(t.grad.detach().numpy(), cmap="coolwarm", norm=norm)
        right.set_xlabel("torch")

        fig.colorbar(image, ax=[left, right], orientation="horizontal")
        plt.show()

In [6]:
n_embed = 10  # the dimensionality of the character embedding vectors
n_hidden = 64  # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)  # for reproducibility
C = torch.randn((vocab_size, n_embed), generator=g)
# Layer 1
W1 = (
    torch.randn((n_embed * block_size, n_hidden), generator=g)
    * (5 / 3)
    / ((n_embed * block_size) ** 0.5)
)
# using b1 just for fun, it's useless because of BN
b1 = torch.randn(n_hidden, generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

# Note: I am initializing many of these parameters in non-standard ways
# because sometimes initializing with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))  # number of parameters in total
for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size  # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

In [8]:
# forward pass, "chunked" into smaller steps that are possible to backward one at a time

emb = C[Xb]  # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)  # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1  # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
# note: Bessel's correction (dividing by n-1, not n)
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact)  # hidden layer
# Linear layer 2
logits = h @ W2 + b2  # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)  # type: ignore
# if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,  # afaik there is no cleaner way
    norm_logits, logit_maxes, logits, h, hpreact, bnraw, bnvar_inv, bnvar, bndiff2,
    bndiff, hprebn, bnmeani, embcat, emb]:  # fmt: skip
    t.retain_grad()
loss.backward()
loss

tensor(3.3451, grad_fn=<NegBackward0>)

In [9]:
# ** Solutions **

# Derivative of a mean is 1/n
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1 / n

# NOTE dlogprobs contains zeros, so dprobs contains a lot of nan's (1/0)
# Not sure if this is an issue (harder to debug?) or if I should set the grads
# to 0 explicitly instead
dprobs = (1 / probs) * dlogprobs

# NOTE: Broadcasting
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)
dcounts_1 = counts_sum_inv * dprobs  # Branch 1

# d(1/x) = -1/x²
dcounts_sum = (-1 / (counts_sum**2)) * dcounts_sum_inv

# counts.sum(1, keepdims=True) is just forwarding the gradients
dcounts_2 = torch.ones_like(counts) * dcounts_sum  # Branch 2
dcounts = dcounts_1 + dcounts_2

# d/dx e^x = e^x
dnorm_logits = norm_logits.exp() * dcounts

dlogits_1 = dnorm_logits.clone()
# NOTE 1: Broadcasting rules requires the sum

# NOTE 2: This should be exactly 0 but only very close to zero here due to
# floating point nuances. The max value is subtracted only for numerical
# stability and should have *NO IMPACT* on the loss. The mean here is ~4.72e-10
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)

# Not sure I completely understand this backprop here. The values are so close
# to zero it doesn't matter anyway.
dlogits_2 = F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

dlogits = dlogits_1 + dlogits_2

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

dhpreact = (1.0 - h**2) * dh

dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)

dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = (-0.5 * (bnvar + 1e-5) ** -1.5) * dbnvar_inv
dbndiff2 = (1.0 / (n - 1)) * torch.ones_like(bndiff2) * dbnvar
dbndiff += (2 * bndiff) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0 / n * (torch.ones_like(hprebn) * dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += demb[k, j]

In [10]:
# Exercise 1: backprop through the whole thing manually,
# back propagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

cmp("logprobs", dlogprobs, logprobs)
cmp("probs", dprobs, probs)
cmp("counts_sum_inv", dcounts_sum_inv, counts_sum_inv)
cmp("counts_sum", dcounts_sum, counts_sum)
cmp("counts", dcounts, counts)
cmp("norm_logits", dnorm_logits, norm_logits)
# Option 1: Exactly equal
cmp("logit_maxes", dlogit_maxes, logit_maxes)
# Option 2: Approx equal mathematically. Semantically correct.
cmp("logit_maxes", torch.zeros_like(logit_maxes), logit_maxes, img=False)
cmp("logits", dlogits, logits)
cmp("h", dh, h)
cmp("W2", dW2, W2)
cmp("b2", db2, b2)  # Can't render (27,)
cmp("hpreact", dhpreact, hpreact)
cmp("bngain", dbngain, bngain)
cmp("bnbias", dbnbias, bnbias)
cmp("bnraw", dbnraw, bnraw)
cmp("bnvar_inv", dbnvar_inv, bnvar_inv)
cmp("bnvar", dbnvar, bnvar)
cmp("bndiff2", dbndiff2, bndiff2)
cmp("bndiff", dbndiff, bndiff)
cmp("bnmeani", dbnmeani, bnmeani)
cmp("hprebn", dhprebn, hprebn)
cmp("embcat", dembcat, embcat)
cmp("W1", dW1, W1)
cmp("b1", db1, b1)
cmp("emb", demb, emb)
cmp("C", dC, C)

logprobs        | exact: True  | approx: True  | maxdiff: 0.0
probs           | exact: True  | approx: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approx: True  | maxdiff: 0.0
counts_sum      | exact: True  | approx: True  | maxdiff: 0.0
counts          | exact: True  | approx: True  | maxdiff: 0.0
norm_logits     | exact: True  | approx: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approx: True  | maxdiff: 0.0
logit_maxes     | exact: False | approx: True  | maxdiff: 5.587935447692871e-09
logits          | exact: True  | approx: True  | maxdiff: 0.0
h               | exact: True  | approx: True  | maxdiff: 0.0
W2              | exact: True  | approx: True  | maxdiff: 0.0
b2              | exact: True  | approx: True  | maxdiff: 0.0
hpreact         | exact: True  | approx: True  | maxdiff: 0.0
bngain          | exact: True  | approx: True  | maxdiff: 0.0
bnbias          | exact: True  | approx: True  | maxdiff: 0.0
bnraw           | exact: True  | approx: True  | max